<a href="https://colab.research.google.com/github/Tikquuss/grokking_beyong_l2_norm/blob/main/matrix_factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ...

In [ ]:
# !git clone https://github.com/Tikquuss/grokking_beyong_l2_norm
# %cd grokking_beyong_l2_norm
# # #! ls
# ! pip install -r requirements.txt
LOG_DIR="/content/LOGS"

In [ ]:
import os
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt

import scipy
from tqdm import tqdm

In [ ]:
from plotters.utils import get_twin_axis, FIGSIZE, LINEWIDTH, FIGSIZE_SMALL, FIGSIZE_LARGE, FIGSIZE_MEDIUM, FONTSIZE, LABEL_FONTSIZE, TICK_LABEL_FONTSIZE, MARKERSIZE
from plotters.img_show import custom_imshow

from matplotlib.lines import Line2D
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
print(default_colors)

In [ ]:
from sparse_recovery.utils import sample_iterations_for_plotting, select_log_space
from sparse_recovery.utils import plot_t1_t2, find_memorization_generalization_steps, find_stable_step_final_value
from sparse_recovery.compressed_sensing import calculate_coherence
from sparse_recovery.matrix_factorization import get_matrices_UV, get_matrices_IndentityUV, get_matrices_QR_UV, calculate_local_coherence
from sparse_recovery.matrix_factorization import get_data_matrix_factorization, solve_matrix_factorization_nuclear_norm
from sparse_recovery.matrix_factorization import get_gradient_matrix_factorization, subgradient_descent_matrix_factorization

# Coherence

In [ ]:
problem = 'matrix-completion'
#problem = 'matrix-sensing'

In [ ]:
n_1, n_2 = 10, 10  # Dimension of A*
n = n_1 * n_2
symmetric=False
r=5
seed = None
N=3
tau=0.2

#A_star, U_star, Sigma_star, V_star = get_matrices_IndentityUV(n_1, n_2, r, distribution="normal", scaler=None, seed=seed)
#A_star, U_star, Sigma_star, V_star = get_matrices_QR_UV(n_1, n_2, r, distribution="normal", scaler=None, seed=seed)
A_star, U_star, Sigma_star, V_star = get_matrices_UV(n_1, n_2, r, symmetric, normalize=True, scale=None,  seed=seed)

#(10, 9) (10, 10) (9,) (9, 9)
print(A_star.shape, U_star.shape, Sigma_star.shape, V_star.shape)

(X1, X2, _, _), _ = get_data_matrix_factorization(A_star, U_star, V_star, N, problem, tau=tau, variance=1/N, seed=seed)

mu = calculate_coherence(A=X1.T, B=U_star)
nu = calculate_coherence(A=X2.T, B=V_star)
print(mu, nu)

mu, nu, P = calculate_local_coherence(A_star, U_n1=None, V_n2=None)
print(max(mu), max(nu), mu, nu)

In [ ]:
from matplotlib.colors import LogNorm
label_fontsize=20
ticklabel_fontsize=15
rows, cols = 1, 2
figsize=FIGSIZE_SMALL
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

ax = fig.add_subplot(rows, cols, 1)
img_data = A_star # (n_1, n_2)
img = custom_imshow(
    img_data, ax=ax, fig=fig, add_text=False,
    hide_ticks_and_labels=False, xticklabels=np.arange(1, img_data.shape[1]+1), yticklabels=np.arange(1, img_data.shape[0]+1),
    filter_step_xticks=5, filter_step_yticks=10, log_x=False, log_y=False, base=10,
    rotation_x=90, rotation_y=0,
    x_label="$j$",  y_label="$i$",
    # Use LogNorm to apply a logarithmic scale
    colormesh_kwarg={"shading":'auto', "cmap":'viridis'},#, 'norm':LogNorm(vmin=img_data.min(), vmax=img_data.max())},
    imshow_kwarg={},
    colorbar=True, colorbar_label=f'$A^*_{{ij}}$',
    label_fontsize=label_fontsize,
    ticklabel_fontsize=ticklabel_fontsize,
    show=False, fileName=None, dpf=None,
    use_imshow=False
)

ax = fig.add_subplot(rows, cols, 2)
img_data = np.array(P) # (alphas, dimensions)
img = custom_imshow(
    img_data, ax=ax, fig=fig, add_text=False,
    hide_ticks_and_labels=False, xticklabels=np.arange(1, img_data.shape[1]+1), yticklabels=np.arange(1, img_data.shape[0]+1),
    filter_step_xticks=5, filter_step_yticks=10, log_x=False, log_y=False, base=10,
    rotation_x=90, rotation_y=0,
    x_label="$\\nu$",  y_label="$\\mu$",
    # Use LogNorm to apply a logarithmic scale
    colormesh_kwarg={"shading":'auto', "cmap":'viridis'},#, 'norm':LogNorm(vmin=img_data.min(), vmax=img_data.max())},
    imshow_kwarg={},
    colorbar=True, colorbar_label=f'$\\mu_i + \\nu_j$',
    label_fontsize=label_fontsize,
    ticklabel_fontsize=ticklabel_fontsize,
    show=False, fileName=None, dpf=None
)

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/coherence_P"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

In [ ]:
A_star, U_star, Sigma_star, V_star = get_matrices_UV(n_1, n_2, r, symmetric, normalize=True, scale=None,  seed=seed)

N_max = n# int(0.9 * n)
all_N = list(range(1, N_max, 1)) # Number of measurements, N < n

all_tau = np.linspace(0, 1, 11)  # Range of tau values to test
all_tau = np.linspace(0, 1, 1*10+1)  # Range of tau values to test
all_tau = np.linspace(0, 1, 2*10+1) # Range of tau values to test

coherences = [ [] for _ in range(len(all_N)) ]
for i, N in tqdm(enumerate(all_N), total=len(all_N)) :
    for j, tau in enumerate(all_tau):
        (X1, X2, _, _), _ = get_data_matrix_factorization(A_star, U_star, V_star, N, problem, tau=tau, variance=1/N, seed=None)

        mu = calculate_coherence(A=X1.T, B=U_star)
        nu = calculate_coherence(A=X2.T, B=V_star)

        coherences[i].append(max(mu, nu))
        #coherences[i].append(np.mean(mu) + np.mean(nu))

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
color_indices = np.linspace(0, 1, len(all_N))
colors = plt.cm.viridis(color_indices)
for i, N in enumerate(all_N):
    ax.plot(all_tau, coherences[i], label=f'N={N}', color=colors[i])
ax.set_xlabel('$\\tau$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('coherence', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
#ax.set_xscale('log')
#ax.set_yscale('log')
#ax.grid()
#ax.legend()
# Create a color bar for the sparsity levels `s`
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_N), vmax=max(all_N)))
sm.set_array([])  # We only need the colormap here, no actual data
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('N', fontsize=LABEL_FONTSIZE)

#plt.savefig(f"{LOG_DIR}/coherence_tau"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

# Convex Programming

In [ ]:
problem = 'matrix-completion'
#problem = 'matrix-sensing'

In [ ]:
n_1, n_2 = 10, 10  # Dimension of A*
n = n_1 * n_2
symmetric=False
r=2
seed = None
N=10
tau = 1.0

A_star, U_star, Sigma_star, V_star = get_matrices_UV(n_1, n_2, rank=r, symmetric=symmetric, normalize=True, scale=None, seed=seed)

(X1, X2, X2_bullet_X1, y_star), (X1_bar, X2_bar, X2_bullet_X1_bar, y_star_bar) = get_data_matrix_factorization(A_star, U_star, V_star, N, problem, tau=tau, variance=1/N, seed=seed)

A = solve_matrix_factorization_nuclear_norm(
    n_1, n_2, y_star,
    X1X2=(X1, X2), X2_bullet_X1=X2_bullet_X1,
    X1X2_bar=(X1_bar, X2_bar), X2_bullet_X1_bar=X2_bullet_X1_bar, EPSILON=1e-6)

recovery_error = np.linalg.norm(A - A_star, ord="fro") / np.linalg.norm(A_star, ord="fro")
print(recovery_error)

In [ ]:
# Parameters
n_1, n_2 = 10, 10  # Dimension of A*
n = n_1 * n_2
symmetric=False
seed = None

#all_s =  [1, 2]
all_s = [s for s in list(range(1, min(n_1, n_2)+10, 1)) if s <= min(n_1, n_2)] # Rank <= min(n_1, n_2)
#all_s = [s for s in list(range(1, min(n_1, n_2)+10, 5)) if s <= min(n_1, n_2)] # Rank <= min(n_1, n_2)

all_N =  [10, 20, 30]
all_N = list(range(1, n+11, 10)) # Number of measurements N
#all_N = list(range(1, n+11, 1))
if problem == "matrix-completion":
    all_N = [N for N in all_N if N <= n]

all_tau = np.arange(10+1)/10  # Range of tau values to test
# all_tau = [0.0]

errors = {}
min_N_for_s = {}

for iii, tau in enumerate(all_tau):
    print(f"tau = {tau}, {iii+1}/{len(all_tau)}")
    errors[tau] = [ [] for _ in range(len(all_s)) ]
    min_N_for_s[tau] = [None for _ in range(len(all_s))]
    for i, s in tqdm(enumerate(all_s), total=len(all_s)):

        A_star, U_star, Sigma_star, V_star = get_matrices_UV(n_1, n_2, rank=s, symmetric=symmetric, normalize=True, scale=None, seed=None)

        for j, N in enumerate(all_N) :
            error_mean = 0
            n_trials = 2
            for k in range(n_trials):
                # Construct a measurements matrix X and output y
                (X1, X2, X2_bullet_X1, y_star), (X1_bar, X2_bar, X2_bullet_X1_bar, y_star_bar) = get_data_matrix_factorization(A_star, U_star, V_star, N, problem, tau=tau, variance=1/N, seed=None)

                # Solve the l*-minimization problem to recover A
                A = solve_matrix_factorization_nuclear_norm(
                    n_1, n_2, y_star,
                    X1X2=(X1, X2), X2_bullet_X1=X2_bullet_X1,
                    X1X2_bar=(X1_bar, X2_bar), X2_bullet_X1_bar=X2_bullet_X1_bar, EPSILON=1e-6)

                recovery_error = np.linalg.norm(A - A_star, ord="fro") / np.linalg.norm(A_star, ord="fro")
                error_mean += recovery_error

            error_mean /= n_trials
            errors[tau][i].append(error_mean)
            if error_mean < 1e-3 and min_N_for_s[tau][i] is None:
                min_N_for_s[tau][i] = N

In [ ]:
from matplotlib.colors import LogNorm

rows, cols = len(all_tau), 3
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

for iii, tau in enumerate(all_tau):
    ax = fig.add_subplot(rows, cols, 1+3*iii)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    color_indices = np.linspace(0, 1, len(all_s))
    colors = plt.cm.viridis(color_indices)
    for i, s in enumerate(all_s):
        ax.plot(all_N, errors[tau][i], label=f'r={s}', color=colors[i])
    ax.set_xlabel('Number of measurements (N)', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel(f'Error ($\\tau$ = {round(tau, 3)})', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    ax.set_yscale('log')
    #ax.grid()
    #ax.legend()
    # Create a color bar for the sparsity levels `s`
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_s), vmax=max(all_s)))
    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('Rank (r)', fontsize=LABEL_FONTSIZE)

    ax = fig.add_subplot(rows, cols, 2+3*iii)
    img_data = np.array(errors[tau]) # (M, N) = (len(all_s), len(all_N))
    img = custom_imshow(
        img_data, ax=ax, fig=fig, add_text=False,
        hide_ticks_and_labels=False, xticklabels=all_N, yticklabels=all_s,
        filter_step_xticks=1, filter_step_yticks=3, log_x=False, log_y=False, base=10,
        rotation_x=90, rotation_y=0,
        x_label="Number of measurements (N)",  y_label="Rank (r)",
        # Use LogNorm to apply a logarithmic scale
        colormesh_kwarg={"shading":'auto', "cmap":'viridis', 'norm':LogNorm(vmin=img_data.min(), vmax=img_data.max())},
        imshow_kwarg={},
        colorbar=True, colorbar_label='Error',
        label_fontsize=label_fontsize,
        ticklabel_fontsize=ticklabel_fontsize,
        show=False, fileName=None, dpf=None
    )

    ax = fig.add_subplot(rows, cols, 3+3*iii)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    ax.plot(all_s, min_N_for_s[tau], label="true")
    #ax.plot(all_s, [s * np.log(n) for s in all_s], label="predict")
    ax.set_xlabel('Rank (r)', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('$N_{min}$ for recovery', fontsize=LABEL_FONTSIZE)
    ax.yaxis.tick_right()  # Move y-axis ticks to the right
    ax.yaxis.set_label_position("right")  # Move y-axis label to the right
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    #ax.legend()
    #ax.grid()

# Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/{problem}_convex_programming_all"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')
#plt.savefig(f"{LOG_DIR}/{problem}_convex_programming"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
color_indices = np.linspace(0, 1, len(all_tau))
colors = plt.cm.viridis(color_indices)
for i, tau in enumerate(all_tau):
    ax.plot(all_s, min_N_for_s[tau], label=f'$\\tau$={round(tau, 3)}', color=colors[i])

ax.set_xlabel('Rank (r)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$N_{min}$ for recovery', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
# Create a color bar
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_tau), vmax=max(all_tau)))
sm.set_array([])  # We only need the colormap here, no actual data
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('$\\tau$', fontsize=LABEL_FONTSIZE)

#plt.savefig(f"{LOG_DIR}/{problem}_convex_programming_N_min"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

# Gradient Descent : Subgradient & Projected & Proximal (ISTA) Gradient Descent

In [ ]:
problem = 'matrix-completion'
# problem = 'matrix-sensing'

In [ ]:
LABELS_PLOTS = {
    'train_errors' : '$||X vec(A^{(t)}) - y^*||_2 \ \ / \ \ ||y^*||_2$',
    #'train_errors' : '$||X a^{(t)} - y^*||_2 \ \ / \ \ ||y^*||_2$',
    'errors' : '$||A^{(t)} - A^*||_F \ \ / \ \ ||A^*||_F$',
}

## Gradients Norms, $A^{(t)}$ norm : Figures 5 (subgradient), 14 (projected subgradient) and 15 (ISTA/Proximal Gradient)

In [ ]:
n_1, n_2 = 10, 10  # Dimension of A*
n = n_1 * n_2
symmetric=False
r=2
seed=0
N=80
tau=0.0

N_min = max(n_1, n_2) * r * np.log(max(n_1, n_2))
print(f"N_min={N_min}")
print(f"N={N}")

A_star, U_star, Sigma_star, V_star = get_matrices_UV(n_1, n_2, rank=r, symmetric=symmetric, normalize=True, scale=None, seed=seed)
(X1, X2, X2_bullet_X1, y_star), (X1_bar, X2_bar, X2_bullet_X1_bar, y_star_bar) = get_data_matrix_factorization(A_star, U_star, V_star, N, problem, tau=tau, variance=1/N, seed=seed)

alpha = 1e-1
beta_star = 1e-4
beta_2 = 0.0
init_scale = 1e-6
init_method = "identity"
# init_method = "random"

method = "subgradient"
# method = "proj_subgradient"
# method = "ISTA"

max_iter=10**7*1+1
# max_iter=10**5+1
# max_iter=10**4*5+1
# max_iter=10**3+1

# T = max_iter/eval_step
# A~(n_1, n_2), errors~(T), train_errors~(T), all_A~(T, n_1, n_2), all_Sigma~(T, min(n_1, n_2)), iterations~(T), A_LS~(n_1,n_2)

A, errors, train_errors, all_A, all_Sigma, iterations, A_LS = subgradient_descent_matrix_factorization(
    X=X2_bullet_X1, X_bar=X2_bullet_X1_bar, y_star=y_star, A_star=A_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_star=beta_star, beta_1=0.0,
    init_scale=init_scale, init_method=init_method,
    max_iter=max_iter,
    eval_first=10**3, eval_step=10**4, tol=1e-6,
    #eval_first=10**3, eval_step=10**1, tol=1e-6,
    verbose=True
)

# fileName = f"matrix_factorization_{problem}_test"
# exp_dir = f"{LOG_DIR}/{fileName}"
# os.makedirs(exp_dir, exist_ok=True)
# np.save(f"{exp_dir}/A_star.npy", A_star)
# np.save(f"{exp_dir}/A.npy", A)
# np.save(f"{exp_dir}/errors.npy", errors)
# np.save(f"{exp_dir}/train_errors.npy", train_errors)
# np.save(f"{exp_dir}/all_A.npy", all_A)
# np.save(f"{exp_dir}/all_Sigma.npy", all_Sigma)
# np.save(f"{exp_dir}/iterations.npy", iterations)
# np.save(f"{exp_dir}/A_LS.npy", A_LS)

In [ ]:
zero_indexes = np.where(Sigma_star == 0)[0]
non_zero_indexes = np.where(Sigma_star != 0)[0]

Sigma_recovered = all_Sigma[-1]
#print(b_star.round(4), b_recovered.round(1))

print(Sigma_star[non_zero_indexes].round(4), Sigma_recovered[non_zero_indexes].round(4))
print(Sigma_star[zero_indexes].round(4), Sigma_recovered[zero_indexes].round(4))

In [ ]:
_, _, norms_grad_ratio = get_gradient_matrix_factorization(X2_bullet_X1, y_star, all_A, beta_star=beta_star, ord=2)

plt.plot(iterations, norms_grad_ratio)
plt.xscale("log")
#plt.yscale("log")

In [ ]:
threshold = 1e-4
t_1, t_2 = find_memorization_generalization_steps(train_errors, test_errors = errors, train_steps=iterations, train_threshold=threshold, test_threshold=threshold)
print(t_1, t_2)

In [ ]:
rows, cols = 1, 3
rows, cols = 3, 1
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=True
log_y=True

all_steps = iterations + []
if log_x : all_steps = np.array(all_steps) + 1

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

if cols==2:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm gradient
ax1.plot(all_steps, norms_grad_ratio, "-", color=color_2, linewidth=LINEWIDTH)

#ax1.set_ylabel('$||\\beta H(A)||_2 \ / \ ||G(A)||_F$', fontsize=LABEL_FONTSIZE)
ax1.set_ylabel('Gradients Ratio', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    Line2D([0], [0], color=color_2, linestyle='-', label='$||\\beta H(A^{(t)})||_F \ \ / \ \ ||G(A^{(t)})||_F$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

if rows==1:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
#if rows==2:
ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm b
B = np.array(all_Sigma) # (n_steps, n)
y = np.linalg.norm(B, ord=1, axis=1)
ax1.plot(all_steps, y, "-", color=color_2, linewidth=LINEWIDTH)

norm_b_star = np.linalg.norm(Sigma_star, ord=1)
ax1.axhline(y=norm_b_star, color=color_2, linestyle='--', alpha=0.8, linewidth=LINEWIDTH)

ax1.set_ylabel('$||A||_*$', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    # Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    # Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    Line2D([0], [0], color=color_2, linestyle='-', label='$||A^{(t)}||_*$'),
    Line2D([0], [0], color=color_2, linestyle='--', label='$||A^*||_*$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

#if log_x : ax1.set_xscale('log')
#if log_y : ax1.set_yscale('log')


##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 3)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
#_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

img_data = np.array(all_Sigma) # (n_steps, r)
R = img_data.shape[1]
color_indices = np.linspace(0, 1, R)
colors = plt.cm.viridis(color_indices)
for i in range(R):
    ax.plot(np.array(iterations)+1, img_data[:,i], '-', color=colors[i])

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=False)

###############

if rows==3:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Singular values', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('symlog')

sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=1, vmax=R))
sm.set_array([])  # We only need the colormap here, no actual data
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label(f'Dimensions ~ $\min(n_1, n_2)$', fontsize=LABEL_FONTSIZE)

ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)
#ax.axhline(y=-0.01, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/{problem}_gradient_norm_a_{method}_5x5"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

## Effect of $\alpha \beta$ : Figures 6 and 29

In [ ]:
n_1, n_2 = 10, 10  # Dimension of A*
n = n_1 * n_2
symmetric=False
r=2
seed = 0
N=80
tau=0.0


N_min = max(n_1, n_2) * r * np.log(max(n_1, n_2))
print(f"N_min={N_min}")
print(f"N={N}")

A_star, U_star, Sigma_star, V_star = get_matrices_UV(n_1, n_2, rank=r, symmetric=symmetric, normalize=True, scale=None, seed=seed)
zero_indexes = np.where(Sigma_star == 0)[0]
non_zero_indexes = np.where(Sigma_star != 0)[0]

(X1, X2, X2_bullet_X1, y_star), (X1_bar, X2_bar, X2_bullet_X1_bar, y_star_bar) = get_data_matrix_factorization(A_star, U_star, V_star, N, problem, tau=tau, variance=None, seed=seed)

In [ ]:
all_alpha = [0.0825, 0.1, 0.75, 0.8250000000000001, 1.0, 1.5]
all_alpha = sorted(all_alpha)

#all_beta_1 = [1e-5, 1e-4]
all_beta_1 = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

beta_2 = 0.0
init_scale = 1e-6
init_method = "identity"
# init_method = "random"

method = "subgradient"
# method = "proj_subgradient"
# method = "ISTA"

max_iter=10**7*1+1

all_errors = {}
all_train_errors = {}
all_iterations = {}
all_A = {}
all_Sigma = {}
all_A_LS = {}
all_A_star = {}

fileName = f"{problem}_alpha_beta_n={n}"
fileName_ = f"{problem}_alpha_beta"

In [ ]:
# for i, alpha in enumerate(all_alpha):
#     #print(f"alpha = {alpha}, {(i+1)}/{len(all_alpha)}")

#     all_errors[alpha] = {}
#     all_train_errors[alpha] = {}
#     all_iterations[alpha] = {}
#     all_A[alpha] = {}
#     all_Sigma[alpha] = {}
#     all_A_LS[alpha] = {}

#     for j, beta_star in enumerate(all_beta_1) :
#         print(f"alpha = {alpha}, {(i+1)}/{len(all_alpha)}, beta_star={beta_star}, {(j+1)}/{len(all_beta_1)}")

#         A, errors, train_errors, all_A_tmp, all_Sigma_tmp, iterations, A_LS = subgradient_descent_matrix_factorization(
#             X=X2_bullet_X1, X_bar=X2_bullet_X1_bar, y_star=y_star, A_star=A_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_star=beta_star,
#             init_scale=init_scale, init_method=init_method,
#             max_iter=max_iter, eval_first=10**3, eval_step=10**4, tol=1e-6
#         )

#         all_errors[alpha][beta_star] = errors
#         all_train_errors[alpha][beta_star] = train_errors
#         all_iterations[alpha][beta_star] = iterations
#         all_A[alpha][beta_star] = all_A_tmp
#         all_Sigma[alpha][beta_star] = all_Sigma_tmp
#         all_A_LS[alpha][beta_star] = A_LS

#         #################

#         exp_dir = f"{LOG_DIR}/{fileName}"
#         exp_dir = f"{LOG_DIR}/{fileName}/alpha={alpha}_beta={beta_star}"
#         os.makedirs(exp_dir, exist_ok=True)
#         np.save(f"{exp_dir}/A_star.npy", A_star)
#         np.save(f"{exp_dir}/A.npy", A)
#         np.save(f"{exp_dir}/errors.npy", errors)
#         np.save(f"{exp_dir}/train_errors.npy", train_errors)
#         np.save(f"{exp_dir}/all_A.npy", all_A_tmp)
#         np.save(f"{exp_dir}/all_Sigma.npy", all_Sigma_tmp)
#         np.save(f"{exp_dir}/iterations.npy", iterations)
#         np.save(f"{exp_dir}/A_LS.npy", A_LS)

#         #################

#         print(f"Final train error: {train_errors[-1]}")
#         print(f"Final error: {errors[-1]}")
#         Sigma_recovered = all_Sigma_tmp[-1]
#         print(Sigma_star[non_zero_indexes].round(4), Sigma_recovered[non_zero_indexes].round(4))
#         print(Sigma_star[zero_indexes].round(4), Sigma_recovered[zero_indexes].round(4))

#         ##################

In [ ]:
for i, alpha in enumerate(all_alpha):
    #print(f"alpha = {alpha}, {(i+1)}/{len(all_alpha)}")

    all_errors[alpha] = {}
    all_train_errors[alpha] = {}
    all_iterations[alpha] = {}
    all_A[alpha] = {}
    all_Sigma[alpha] = {}
    all_A_LS[alpha] = {}
    all_A_star[alpha] = {}

    for j, beta_star in enumerate(all_beta_1) :
        print(f"alpha = {alpha}, {(i+1)}/{len(all_alpha)}, beta_star={beta_star}, {(j+1)}/{len(all_beta_1)}")

        #################

        exp_dir = f"{LOG_DIR}/{fileName}/{fileName_}_alpha={alpha}_beta={beta_star}"

        #################

        A_star_  = np.load(f"{exp_dir}/A_star.npy")
        A_ = np.load(f"{exp_dir}/A.npy")

        all_errors[alpha][beta_star] = np.load(f"{exp_dir}/errors.npy")
        all_train_errors[alpha][beta_star] = np.load(f"{exp_dir}/train_errors.npy")
        all_iterations[alpha][beta_star] = np.load(f"{exp_dir}/iterations.npy")
        all_A[alpha][beta_star] = np.load(f"{exp_dir}/all_A.npy")
        all_Sigma[alpha][beta_star] = np.load(f"{exp_dir}/all_Sigma.npy")
        all_A_LS[alpha][beta_star] = np.load(f"{exp_dir}/A_LS.npy")
        all_A_star[alpha][beta_star] = A_star_

        ##################

### Figure 29

In [ ]:
all_beta_star = all_beta_1

In [ ]:
L=len(all_alpha)
cols = min(3, L)
rows = L // cols + 1 * (L % cols != 0)
figsize=(8, 4)
#figsize=(8, 6)
#figsize=(15, 10)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=True
log_y=True

for i, alpha in enumerate(all_alpha):

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    #_, ax, ax1 = get_twin_axis(ax=ax, no_twin=False)

    ax.set_title(f'$\\alpha={round(alpha, 4)}$', fontsize=LABEL_FONTSIZE)

    color_indices = np.linspace(0, 1, len(all_beta_1)+1*0)
    colors = plt.cm.viridis(color_indices)

    for j, beta_1 in enumerate(all_beta_star) :

        all_steps = all_iterations[alpha][beta_1]
        if log_x : all_steps = np.array(all_steps) + 1
        ax.plot(all_steps, all_errors[alpha][beta_1], '--', color=colors[j], linewidth=LINEWIDTH)
        ax.plot(all_steps, all_train_errors[alpha][beta_1], '-', label=f'$\beta$={beta_1}', color=colors[j], linewidth=LINEWIDTH)

        # Plot t_2
        t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[alpha][beta_1], K=3, tolerance_fraction=0.05, M=2)
        t = t_2
        if t is not None :
            ax.axvline(x=t, ymin=0.01, ymax=1., color=colors[j], linestyle='--', lw=1.)
            ax.plot([t, t], [0, 0], 'o', color='b')

        # B = np.array(all_b[alpha][beta_1]) # (n_steps, n)
        # y = np.linalg.norm(B, ord=1, axis=1)
        # ax1.plot(all_steps, y, "-.", color=colors[j], linewidth=1.0)

    #ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)

    if (rows-1)*cols <= i < rows*cols : ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
    if i%cols==0 : ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)

    ########### Color bar

    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_beta_1), vmax=max(all_beta_1)))
    import matplotlib.colors as mcolors
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=mcolors.LogNorm(vmin=min(all_beta_1), vmax=max(all_beta_1)))

    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(f'$\\beta$', fontsize=LABEL_FONTSIZE)
    # Set the ticks to correspond to the values in `all_beta_1`
    #cbar.set_ticks(all_beta_1)  # Sets tick positions based on `all_beta_1`
    #cbar.set_ticklabels([str(beta_1) for beta_1 in all_beta_1])  # Sets tick labels to match `all_beta_1`

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    legend_elements = [
        Line2D([0], [0], color='k', linestyle='-', label=LABELS_PLOTS['train_errors']),
        Line2D([0], [0], color='k', linestyle='--', label=LABELS_PLOTS['errors'])
        ]
    ax.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

    # break

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/{problem}_scaling_alpha_and_beta_star_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 6

In [ ]:
# from collections import defaultdict

all_alpha_beta_1 = []
all_test_errors = []
all_t2 = []

all_test_errors_predict = []
all_t2_predict = []

for i, alpha in enumerate(all_alpha):
    for j, beta_1 in enumerate(all_beta_1) :

        all_steps = all_iterations[alpha][beta_1]
        t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[alpha][beta_1], K=3, tolerance_fraction=0.05, M=2)
        #t_1, t_1_index = find_stable_step_final_value(all_steps, all_train_errors[alpha][beta_1], K=3, tolerance_fraction=0.05, M=2)

        if t_2 is None :
            t_2 = all_steps[-1]
            t_2_index = all_steps.index(t_2)

        #
        all_alpha_beta_1.append(alpha * beta_1)

        # from relative to absolute error
        #error = all_errors[alpha][beta_1][t_2_index] * np.linalg.norm(A_star, 'fro') / n

        A_star = all_A_star[alpha][beta_1]
        A_t2 = all_A[alpha][beta_1][t_2_index]
        error =  np.linalg.norm(A_t2-A_star, ord="nuc") / n**0.5
        all_test_errors.append(error)


        all_test_errors_predict.append(alpha * beta_1)

        all_t2.append(t_2)

        A_LS = all_A_LS[alpha][beta_1]
        #A_LS = all_A[alpha][beta_1][all_steps.index(t_1)]
        t_2_predict = np.linalg.norm(A_LS-A_star, ord=2) / (alpha * beta_1)
        all_t2_predict.append(t_2_predict )

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=True
log_y=True

#############################################################

ax = fig.add_subplot(rows, cols, 1)
#_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_alpha_beta_1, all_t2, "o", color=color_1, label="$t_2$", markersize=MARKERSIZE)
ss = "$\\frac{ ||\hat{A}-A^*||_{2 \\to 2} }{ \\alpha \\beta }$"
#ss = "$||\hat{b}||_{\infty} / \\alpha \\beta_1$"
ax.plot(all_alpha_beta_1, all_t2_predict, '*', color=color_1, label=f"{ss}", markersize=MARKERSIZE)

ax.set_xlabel('$\\alpha \\beta$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t$', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

#ax.legend(fontsize=LABEL_FONTSIZE*0.8)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### errors

ax1.plot(all_alpha_beta_1, all_test_errors, "s", color=color_2, label="$\\frac{1}{n}||A^{(t_2)} - A^*||_*$", markersize=MARKERSIZE)
ax1.plot(all_alpha_beta_1, all_test_errors_predict, 'X', color=color_2, label=f"$\\alpha \\beta$", markersize=MARKERSIZE)


#ax1.set_xlabel('$\\alpha \\beta_1$', fontsize=LABEL_FONTSIZE)
ax1.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax1.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

#if log_x : ax1.set_xscale('log')
if log_y : ax1.set_yscale('log')

#############################################################

# legend_elements = [
#     Line2D([0], [0], color=color_1, linestyle='.', label='$t_2$'),
#     Line2D([0], [0], color=color_1, linestyle='x', label=f'{ss}'),
#     Line2D([0], [0], color=color_2, linestyle='.', label='$||A^{(t_2)} - A^*||_F$'),
#     Line2D([0], [0], color=color_2, linestyle='x', label='$\\alpha \\beta_1$')
#     ]

# legend_elements = [
#     Line2D([0], [0], color=color_1, marker='o', linestyle='None', markerfacecolor=color_1, label='$t_2$', markersize=10),
#     Line2D([0], [0], color=color_1, marker='*', linestyle='None', markerfacecolor=color_1, label=f'{ss}', markersize=10),
#     Line2D([0], [0], color=color_2, marker='s', linestyle='None', markerfacecolor=color_2, label='$||A^{(t_2)} - A^*||_F$', markersize=10),
#     Line2D([0], [0], color=color_2, marker='X', linestyle='None', markerfacecolor=color_2, label='$\\alpha \\beta_*$', markersize=10),
# ]
#ax1.legend(handles=legend_elements, loc='best', fontsize=LABEL_FONTSIZE*0.6)

handles, labels = ax.get_legend_handles_labels()
handles1, labels1 = ax1.get_legend_handles_labels()
ax1.legend(handles + handles1, labels + labels1, loc='best', fontsize=LABEL_FONTSIZE*0.8)

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/{problem}_scaling_time_and_error_vs_alpha_and_beta_star_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

## Scaling wrt $\tau$ : Figures 8 and 23

In [ ]:
n_1, n_2 = 10, 10  # Dimension of A*
n = n_1 * n_2
symmetric=False
r=2
seed = 0
tau=0.0

N_min = max(n_1, n_2) * r * np.log(max(n_1, n_2))
print(f"N_min={N_min}")

A_star, U_star, Sigma_star, V_star = get_matrices_UV(n_1, n_2, rank=r, symmetric=symmetric, normalize=True, scale=None, seed=seed)

alpha = 1e-0 # 1e-1
beta_star = 1e-5
beta_2 = 0.0
init_scale = 1e-6
init_method = "identity"
# init_method = "random"

max_iter=10**7+1

all_errors = {}
all_train_errors = {}
all_iterations = {}
all_A = {}
all_Sigma = {}
all_A_LS = {}
all_X = {}

all_N =  [70]

all_tau = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#all_tau = [0.0, 0.1, 0.5, 0.8, 1.0]

fileName = f"{problem}_tau={n}"

for i, N in enumerate(all_N):

    all_errors[N] = {}
    all_train_errors[N] = {}
    all_iterations[N] = {}
    all_A[N] = {}
    all_Sigma[N] = {}
    all_A_LS[N] = {}
    all_X[N] = {}

    for j, tau in enumerate(all_tau):
        print(f"N = {N}, {(i+1)}/{len(all_N)}, tau = {tau}, {(j+1)}/{len(all_tau)}")

        (X1, X2, X2_bullet_X1, y_star), (X1_bar, X2_bar, X2_bullet_X1_bar, y_star_bar) = get_data_matrix_factorization(A_star, U_star, V_star, N, problem, tau=tau, variance=None, seed=seed)

        A, errors, train_errors, all_A_tmp, all_Sigma_tmp, iterations, A_LS = subgradient_descent_matrix_factorization(
            X=X2_bullet_X1, X_bar=X2_bullet_X1_bar, y_star=y_star, A_star=A_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_star=beta_star,
            init_scale=init_scale, init_method=init_method,
            max_iter=max_iter, eval_first=10**3, eval_step=10**4, tol=1e-6
        )

        all_errors[N][tau] = errors
        all_train_errors[N][tau] = train_errors
        all_iterations[N][tau] = iterations
        all_A[N][tau] = all_A_tmp
        all_Sigma[N][tau] = all_Sigma_tmp
        all_A_LS[N][tau] = A_LS
        all_X[N][tau] = [(X1, X2, X2_bullet_X1, y_star), (X1_bar, X2_bar, X2_bullet_X1_bar, y_star_bar)]


        #################

        exp_dir = f"{LOG_DIR}/{fileName}"
        exp_dir = f"{LOG_DIR}/{fileName}/N={N}_tau={tau}"
        os.makedirs(exp_dir, exist_ok=True)
        np.save(f"{exp_dir}/A_star.npy", A_star)
        np.save(f"{exp_dir}/A.npy", A)
        np.save(f"{exp_dir}/errors.npy", errors)
        np.save(f"{exp_dir}/train_errors.npy", train_errors)
        np.save(f"{exp_dir}/all_A.npy", all_A_tmp)
        np.save(f"{exp_dir}/all_Sigma.npy", all_Sigma_tmp)
        np.save(f"{exp_dir}/iterations.npy", iterations)
        np.save(f"{exp_dir}/A_LS.npy", A_LS)

        #################

        print(f"Final train error: {train_errors[-1]}")
        print(f"Final error: {errors[-1]}")
        Sigma_recovered = all_Sigma_tmp[-1]
        print(Sigma_star[non_zero_indexes].round(4), Sigma_recovered[non_zero_indexes].round(4))
        print(Sigma_star[zero_indexes].round(4), Sigma_recovered[zero_indexes].round(4))

        ##################

In [ ]:
L=len(all_N)
cols = min(2, L)
rows = L // cols + 1 * (L % cols != 0)

figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=True
log_y=True

for i, N in enumerate(all_N) :

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    #_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    ax.set_title(f"N={N}", fontsize=LABEL_FONTSIZE)

    color_indices = np.linspace(0, 1, len(all_tau)+1*0)
    colors = plt.cm.viridis(color_indices)

    for j, tau in enumerate(all_tau):
        all_steps = all_iterations[N][tau]
        if log_x : all_steps = np.array(all_steps)+1
        ax.plot(all_steps, all_errors[N][tau], '--', color=colors[j], linewidth=LINEWIDTH)
        ax.plot(all_steps, all_train_errors[N][tau], '-', label=f'$\tau$={tau}', color=colors[j], linewidth=LINEWIDTH)

        t=None
        #t=all_t1[s][j]
        if t is not None :
            ax.axvline(x=t, ymin=0.01, ymax=1., color=colors[j], linestyle='--', lw=1.)
            ax.plot([t, t], [0, 0], 'o', color='b')

    ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    ########### Color bar
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_tau), vmax=max(all_tau)))
    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(f'$\\tau$', fontsize=LABEL_FONTSIZE)
    # Set the ticks to correspond to the values in `all_tau`
    cbar.set_ticks(all_tau)  # Sets tick positions based on `all_tau`
    cbar.set_ticklabels([str(tau) for tau in all_tau])  # Sets tick labels to match `all_tau`

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    legend_elements = [
        Line2D([0], [0], color='k', linestyle='-', label=LABELS_PLOTS["train_errors"]),
        Line2D([0], [0], color='k', linestyle='--', label=LABELS_PLOTS["errors"])
        ]
    ax.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
# plt.savefig(f"{LOG_DIR}/{problem}_scaling_N_and_tau_{method}_n={n}_r={r}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')
# #plt.savefig(f"{LOG_DIR}/{problem}_scaling_N_and_tau_{method}_n={n}_r={s}_nologx"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
all_t1 = {}
all_t2 = {}
all_Delta_t = {}

all_train_errors_N_tau = {}
all_test_errors_N_tau = {}

for i, N in enumerate(all_N) :

    all_t1[N] = []
    all_t2[N] = []
    all_Delta_t[N] = []
    all_train_errors_N_tau[N] = []
    all_test_errors_N_tau[N] = []

    for j, tau in enumerate(all_tau):

        all_steps = all_iterations[N][tau]

        t_1, t_1_index = find_stable_step_final_value(all_steps, all_train_errors[N][tau], K=3, tolerance_fraction=0.05, M=2)
        if t_1 is None :
            t_1 = all_steps[-1]
            t_1_index = all_steps.index(t_2)

        t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[N][tau], K=3, tolerance_fraction=0.05, M=2)
        if t_2 is None :
            t_2 = all_steps[-1]
            t_2_index = all_steps.index(t_2)


        all_t1[N].append(t_1)
        all_t2[N].append(t_2)
        all_Delta_t[N].append(t_2 - t_1)
        all_train_errors_N_tau[N].append(all_train_errors[N][tau][t_1_index])
        all_test_errors_N_tau[N].append(all_errors[N][tau][-1])

In [ ]:
L=len(all_N)
cols = min(2, L)
rows = L // cols + 1 * (L % cols != 0)

figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=False
log_y=True

for i, N in enumerate(all_N) :

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    _, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    ax.set_title(f"N={N}", fontsize=LABEL_FONTSIZE)

    # ax.plot(all_tau, all_t1[N], "o", color=color_1, label="$t_1$", markersize=MARKERSIZE)
    # ax.plot(all_tau, all_t2[N], "*", color=color_1, label="$t_2$", markersize=MARKERSIZE)

    ax.plot(all_tau, all_t1[N], "-", color=color_1, label="$t_1$", linewidth=LINEWIDTH)
    ax.plot(all_tau, all_t2[N], "--", color=color_1, label="$t_2$", linewidth=LINEWIDTH)


    ax.set_xlabel('$\\tau$', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('$t$', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
    ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)
    # Set the ticks to correspond to the values in `all_tau`
    ax.set_xticks(all_tau)  # Sets tick positions based on `all_tau`
    ax.set_xticklabels([str(tau) for tau in all_tau])  # Sets tick labels to match `all_tau`

    #ax.legend(fontsize=LABEL_FONTSIZE*0.8)

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    ##################### errors


    # ax1.plot(all_L, all_train_errors_N_tau[N], 's', color=color_2, label='$||X vec(A^{(t_1)}) - y^*||_F \ \ / \ \ ||y^*||_2$', markersize=MARKERSIZE)
    # ax1.plot(all_L, all_test_errors_N_tau[N], "X", color=color_2, label='$||A^{(t_2)} - A^*||_F \ \ / \ \ ||A^*||_F$', markersize=MARKERSIZE)

    ax1.plot(all_tau, all_train_errors_N_tau[N], '-', color=color_2, label='$||X vec(A^{(t_1)}) - y^*||_F \ \ / \ \ ||y^*||_2$', linewidth=LINEWIDTH)
    ax1.plot(all_tau, all_test_errors_N_tau[N], "--", color=color_2, label='$||A^{(t_2)} - A^*||_F \ \ / \ \ ||A^*||_F$', linewidth=LINEWIDTH)

    #ax1.set_xlabel('$\\tau$', fontsize=LABEL_FONTSIZE)
    ax1.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
    ax1.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
    ax1.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

    #if log_x : ax1.set_xscale('log')
    if log_y : ax1.set_yscale('log')

    #############################################################

    handles, labels = ax.get_legend_handles_labels()
    handles1, labels1 = ax1.get_legend_handles_labels()
    ax1.legend(handles + handles1, labels + labels1, loc='best', fontsize=LABEL_FONTSIZE*0.8)

    #break

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/{problem}_scaling_N_vs_N_tau_large_{method}_n={n}_r={r}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

In [ ]:
rows, cols = 2, 2
figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=False
log_y=True

color_indices = np.linspace(0, 1, len(all_tau)+1*0)
colors = plt.cm.viridis(color_indices)

##################### test errors
ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, tau in enumerate(all_tau) :
    #yy = [all_train_errors_N_L[N][j] for N in all_N]
    yy = [min(all_train_errors[N][tau]) for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"$\\tau={tau}$", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||X vec(A^{(t_1)}) - y^*||_F \ \ / \ \ ||y^*||_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### test errors
ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, tau in enumerate(all_tau) :
    #yy = [all_test_errors_N_L[N][j] for N in all_N]
    yy = [min(all_errors[N][tau]) for N in all_N]
    ax.plot(all_N, yy, '--', color=colors[j], label=f"$\\tau={tau}$", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||A^{(t_2)} - A^*||_F \ \ / \ \ ||A^*||_F$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### t_1
ax = fig.add_subplot(rows, cols, 3)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, tau in enumerate(all_tau) :
    yy = [all_t1[N][j] for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"$\\tau={tau}$", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t_1$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### t_2
ax = fig.add_subplot(rows, cols, 4)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, tau in enumerate(all_tau) :
    yy = [all_t2[N][j] for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"$\\tau={tau}$", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

#####################

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

# ##
#plt.savefig(f"{LOG_DIR}/{problem}_scaling_N_vs_N_tau_small_{method}_n={n}_r={s}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()